In [1]:
!pip install pydeck
!pip install ortools

In [179]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import pydeck as pdk
import requests
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

import warnings
warnings.filterwarnings("ignore")


source data: https://www.getthedata.com/open-pubs/manchester

In [217]:
pubs = pd.read_csv('open_pubs.csv')
pubs.head()

,name,address,postcode,latitude,longitude
0,Afflecks & Brown,"4 Hilton Street, Manchester",M4 1NB,53.483184,-2.235417
1,Alvarium,"Dorsey House, 8 Dorsey Street, Manchester",M4 1LU,53.483875,-2.235012
2,Atlas Bar Cafe,"376 Deansgate, Manchester",M3 4LY,53.474988,-2.251869
3,Avan Lounge & Bar,"Unit 4, 165 Great Ducie Street, Manchester",M3 1FF,53.491721,-2.248791
4,Band on the Wall,"25-27 Swan Street, Manchester",M4 5JZ,53.485073,-2.233933


In [218]:
pubs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       105 non-null    object
 1   address    105 non-null    object
 2   postcode   105 non-null    object
 3   latitude   105 non-null    object
 4   longitude  105 non-null    object
dtypes: object(5)
memory usage: 4.2+ KB


There are three records without longitude/ latitude coordinates - remove them from data set.

In [219]:
pubs = pubs[pubs["longitude"] != '\\N'].reset_index()
pubs["latitude"] = pubs["latitude"].astype(float)
pubs["longitude"] = pubs["longitude"].astype(float)
pubs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   index      102 non-null    int64  
 1   name       102 non-null    object 
 2   address    102 non-null    object 
 3   postcode   102 non-null    object 
 4   latitude   102 non-null    float64
 5   longitude  102 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 4.9+ KB


I want to explore the city and don't want the solver to give a pub crawl where the pubs are all next door to one another. To prevent this, I apply a crude approach of rounding longitude and latitude coordinates and dropping duplicates so that only one pub will be considered within a given area.

In [220]:
pubs["latitude_rnd"] = pubs["latitude"].round(3)
pubs["longitude_rnd"] = pubs["longitude"].round(3)
pubs = pubs.drop_duplicates(subset=["latitude_rnd", "longitude_rnd"]).reset_index()
pubs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   level_0        50 non-null     int64  
 1   index          50 non-null     int64  
 2   name           50 non-null     object 
 3   address        50 non-null     object 
 4   postcode       50 non-null     object 
 5   latitude       50 non-null     float64
 6   longitude      50 non-null     float64
 7   latitude_rnd   50 non-null     float64
 8   longitude_rnd  50 non-null     float64
dtypes: float64(4), int64(2), object(3)
memory usage: 3.6+ KB


## Plot Points of Interest
As well as the pubs, I also add coordinates for my hotel and some points of interest that I would like to visit

In [221]:
hotel = pd.DataFrame(
    [
       {
        "name" : "Kimpton Clocktower Hotel",
        "address" : "Oxford Street, Manchester",
        "postcode" : "M60 7HA",
        "latitude" : 53.4744,
        "longitude" : -2.2404
        } 
    ]
    
)

attractions = pd.DataFrame(
    [
       {
        "name" : "Alan Turing Memorial",
        "address" : "Sackville Gardens, Manchester",
        "postcode" : "M1 3HB",
        "latitude" : 53.474584,
        "longitude" : -2.234793 
        } ,
        
        {
        "name" : "National Football Museum",
        "address" : "Todd Street, Manchester",
        "postcode" : "M4 3BG",
        "latitude" : 53.485340,
        "longitude" : -2.242260
        } ,
        
        {
        "name" : "John Rylands Library",
        "address" : "150 Deansgate, Manchester",
        "postcode" : "M3 3EH",
        "latitude" : 53.480312,
        "longitude" : -2.248746
        } ,
        
        {
        "name" : "Greater Manchester Police Museum",
        "address" : "57A Newton St, Manchester",
        "postcode" : "M1 1ET",
        "latitude" : 53.482451,
        "longitude" : -2.232211
        }  
    ]
    
)

view_state = pdk.ViewState(latitude=53.483959, longitude=-2.244644, zoom=13, pitch=0)

pub_layer = pdk.Layer(
    "ScatterplotLayer",
    data=pubs,
    get_position='[longitude, latitude]',
    get_color=[255,0,0],
    get_radius=10,
    pickable=True
)

hotel_layer = pdk.Layer(
    "ScatterplotLayer",
    data=hotel,
    get_position='[longitude, latitude]',
    get_color=[0,255,0],
    get_radius=20,
    pickable=True
)

attractions_layer = pdk.Layer(
    "ScatterplotLayer",
    data=attractions,
    get_position='[longitude, latitude]',
    get_color=[0,0,255],
    get_radius=20,
    pickable=True
)

layers = [pub_layer, hotel_layer, attractions_layer]

deck = pdk.Deck(layers=layers, 
                initial_view_state=view_state, 
                #map_style="mapbox://styles/mapbox/light-v9",
                tooltip={"text":"{name}"}
               )

pdk.settings.json_renderer = "notebook"
deck.show()


## Create Parameters
- I will stop for 30 minutes in each pub and have one pint
- I will apply a fixed penalty for each pub not visited
- Each attraction has a specific stop time and penalty

In [247]:
# pub params
pubs["type"] = "pub"
pubs["pints"] = 1
pubs["stop_time"] = 30
pubs["penalty"] = 50

# hotel params
hotel["type"] = "hotel"
hotel["pints"] = 0
hotel["stop_time"] = 0
hotel["penalty"] = 0

# attractions params
attractions["type"] = "attraction"
attractions["pints"] = 0
attractions["stop_time"] = [10, 90, 60, 60]
attractions["penalty"] = [60, 120, 120, 70]

# merge all points into a single df
nodes = pd.concat([hotel, pubs, attractions], ignore_index=True)

# extract location coordinates
coords = nodes[['longitude', 'latitude']].values.tolist()
coord_str = ';'.join([f"{lon},{lat}" for lon, lat in coords])
# OSRM table API (walking profile)
url = f"http://router.project-osrm.org/table/v1/foot/{coord_str}?annotations=distance"
response = requests.get(url)
data = response.json()

# Extract distance matrix (in metres)
distance_matrix = data['distances']

# convert to time basis assuming pace of 5 km/h
time_matrix = [[round((dist/1000) * (60/5)) for dist in row] for row in distance_matrix]



## Solve Travelling Salesman Problem
Minimise objective function subject to constraints:
- The pub crawl will last 8 hours (480 mins)
- I will start and finish at my hotel
- I have a maximum capacity of 8 pints

In [246]:
def create_data_model():
    data = {}
    data["time_matrix"] = time_matrix
    data["depot"] = 0
    data["pints"] = nodes["pints"].tolist()
    data["pint_capacity"] = [8]
    data["stop_times"] = nodes["stop_time"].tolist()
    data["time_capacity"] = 480
    data["latitude"] = nodes["latitude"].tolist()
    data["longitude"] = nodes["longitude"].tolist()
    data["penalty"] = nodes["penalty"].tolist()
    return data 


def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()} mins")
    index = routing.Start(0)
    route_time = 0
    plan_output = ""
    while not routing.IsEnd(index):
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_time += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += f"Route time: {route_time} mins\n"
    print(plan_output)


def extract_route(manager, routing, solution, nodes, data):
    """Extract the route into a DataFrame."""
    route = []
    index = routing.Start(0)
    while not routing.IsEnd(index):
        node_id = manager.IndexToNode(index)
        node_info = nodes.iloc[node_id]
        route.append({
            "order": len(route),
            "node_id": node_id,
            "name": node_info["name"],
            "type": node_info["type"],
            "pints": node_info["pints"],
            "stop_time": node_info["stop_time"],
            "latitude": node_info["latitude"],
            "longitude": node_info["longitude"]
        })
        index = solution.Value(routing.NextVar(index))

    # Add the final depot
    node_id = manager.IndexToNode(index)
    node_info = nodes.iloc[node_id]
    route.append({
        "order": len(route),
        "node_id": node_id,
        "name": node_info["name"],
        "type": node_info["type"],
        "pints": node_info["pints"],
        "stop_time": node_info["stop_time"],
        "latitude": node_info["latitude"],
        "longitude": node_info["longitude"]
    })

    return pd.DataFrame(route)


# Instantiate the data problem.
data = create_data_model()

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(
    len(data["time_matrix"]), 1, data["depot"] # 1 = num vehicles (1 for TSP)
)

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

def time_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    travel_time = data["time_matrix"][from_node][to_node]
    stop_time = data["stop_times"][from_node]
    value = travel_time + stop_time
    return int(value)

transit_callback_index = routing.RegisterTransitCallback(time_callback)
    
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    data["time_capacity"],  # maximum time per vehicle
    True,  # start cumul to zero
    "Time"
)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
# Add capacity constraint
def pints_callback(from_index):
    from_node = manager.IndexToNode(from_index)
    return data["pints"][from_node]
    
pints_callback_index = routing.RegisterUnaryTransitCallback(pints_callback)
routing.AddDimensionWithVehicleCapacity(
    pints_callback_index,
    0,
    data["pint_capacity"],
    True,
    "Capacity"
)

# Allow to drop nodes  
for node in range(1, len(data["time_matrix"])):  # skip depot
    penalty_value = data["penalty"][node]
    routing.AddDisjunction([manager.NodeToIndex(node)], penalty_value)

                      
# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.CHRISTOFIDES
)
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
)
search_parameters.time_limit.seconds = 10
        
#search_parameters.time_limit.seconds = 10

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if solution:
    print_solution(manager, routing, solution)
    df = extract_route(manager, routing, solution, nodes, data)
    print(df[["order", "name", "type", "stop_time"]])


Objective: 2640 mins
Route time: 470 mins

    order                                             name        type  \
0       0                         Kimpton Clocktower Hotel       hotel   
1       1                             John Rylands Library  attraction   
2       2                                           Bar 11         pub   
3       3                             Bridge Street Tavern         pub   
4       4                                        Micro Bar         pub   
5       5                         National Football Museum  attraction   
6       6                                  Boom Battle Bar         pub   
7       7                                        Lono Cove         pub   
8       8                                       Fierce Bar         pub   
9       9  Common also T/as Nell's Ice Cream & Nells Pizza         pub   
10     10                                 Afflecks & Brown         pub   
11     11                             Alan Turing Memorial  attractio

## Route Map

In [250]:
def get_osrm_route(coords, mode="foot", timeout=10):
    
    if len(coords) < 2:
        return None

    full = []
    for a, b in zip(coords[:-1], coords[1:]):
        seg_url = (
            f"http://router.project-osrm.org/route/v1/{mode}/"
            f"{a[0]},{a[1]};{b[0]},{b[1]}"
            f"?overview=full&geometries=geojson&alternatives=false&steps=false"
        )
        resp = requests.get(seg_url, timeout=timeout)
        if resp.status_code != 200:
            print(f"OSRM HTTP {resp.status_code} for segment: {seg_url}")
            return None
        data = resp.json()
        if not data.get("routes"):
            print(f"No route for segment: {a} -> {b}")
            return None

        coords_seg = data["routes"][0]["geometry"]["coordinates"]  # [[lon, lat], ...]
        if not full:
            full.extend(coords_seg)
        else:
            # Avoid duplicating the joint point between segments
            if full[-1] == coords_seg[0]:
                full.extend(coords_seg[1:])
            else:
                full.extend(coords_seg)
    return full


coords = [(row["longitude"], row["latitude"]) for _, row in df.iterrows()]
route_coords = get_osrm_route(coords)
    
# Create a single path object
path_data = pd.DataFrame([{
    "path": route_coords,
    "name": "Walking Route"
}])

lats = [lat for _, lat in route_coords]
lons = [lon for lon, _ in route_coords]

# Start and end points
points_df = pd.DataFrame([
    {
        "order": i,
        "lat": lat,
        "lon": lon
    }
    for i, (lon, lat) in enumerate(coords)
    ])

points_df = pd.merge(
    points_df,
    df,
    how="left",
    on="order"
)

# set colour
colour_map = {
    "pub": [255, 0, 0],
    "hotel": [0, 255, 0],
    "attraction": [0, 0, 255]
}

points_df["colour"] = points_df["type"].map(colour_map)

# Define view state
view_state = pdk.ViewState(
    latitude=np.mean(lats),
    longitude=np.mean(lons),
    zoom=13,
    pitch=0
)

points_layer = pdk.Layer(
        "ScatterplotLayer",
        data=points_df,
        get_position='[lon, lat]',
        get_color="colour",
        get_radius=10,
        pickable=True
    )

# Route layer using PathLayer
route_layer = pdk.Layer(
        "PathLayer",
        data=path_data,
        get_path="path",
        get_color=[255, 165, 0], # orange
        width_scale=10,
        width_min_pixels=2,
        pickable=True
    )

layers = [route_layer, points_layer]

# Combine layers
deck = pdk.Deck(
        layers=layers,
        initial_view_state=view_state,
        tooltip={"text": "{name}"}
    )

pdk.settings.json_renderer = "notebook"
deck.show()
